In [1]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time

In [2]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
len(working_df)

38556

In [5]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)               
    return working_df

In [23]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3
hashtagsall = list(set(hashtagsall))

In [32]:
hashtags=['#RapiersReady']
for hashtag in hashtags:
    print(hashtag)
    try:
        working_df=pd.read_csv(f'tweeterhashtags/{hashtag}.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(60)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)


#RapiersReady
0
0
Series([], Freq: D, Name: hashtags, dtype: int64)


In [24]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'tweeterhashtags/{hashtag}.csv')], ignore_index=True)
    except:
        pass
working_df

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtags,tweet_df
0,1.663155e+18,2023-05-29 12:06:51+00:00,Locklyle_fan,RT @nelsonsdragon: Loved this edit since this ...,0.0,7.0,#AppleTVforLockwoodandCo,NaN
1,1.662960e+18,2023-05-28 23:13:22+00:00,Lockwood1494,RT @pao_morquecho: 💙🖤⚔️ #LockwoodandCoSeason2 ...,0.0,1.0,#AppleTVforLockwoodandCo,NaN
2,1.662960e+18,2023-05-28 23:12:20+00:00,pao_morquecho,💙🖤⚔️ #LockwoodandCoSeason2 #renewlockwoodandco...,1.0,1.0,#AppleTVforLockwoodandCo,NaN
3,1.662960e+18,2023-05-28 23:12:11+00:00,ali_coppola,RT @vansunshin3: has this been done yet? \n#Sa...,0.0,64.0,#AppleTVforLockwoodandCo,NaN
4,1.662833e+18,2023-05-28 14:46:06+00:00,cxpectations,@evmcconnell I had that yesterday!!!😋 (bought ...,0.0,0.0,#AppleTVforLockwoodandCo,NaN
...,...,...,...,...,...,...,...,...
86737,1.660000e+18,2023-05-18 06:15:41+00:00,CreatureBuff,This article sums up the whole mess quite nice...,10.0,3.0,#HauntedbyaType3,NaN
86738,1.660000e+18,2023-05-18 06:01:38+00:00,CreatureBuff,WE WON'T LET YOU! #SaveLockwoodandCo \n\n#Haun...,6.0,1.0,#HauntedbyaType3,NaN
86739,1.660000e+18,2023-05-18 05:52:29+00:00,CreatureBuff,@hollowgirlhan Classic Rock\n\n#HauntedbyaType...,0.0,0.0,#HauntedbyaType3,NaN
86740,1.660000e+18,2023-05-18 05:46:09+00:00,CreatureBuff,"Hey @carlyleandskull, today's hashtag is just ...",13.0,2.0,#HauntedbyaType3,NaN


In [25]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '')
working_df['full_text']

0        RT @nelsonsdragon: Loved this edit since this ...
1        RT @pao_morquecho: 💙🖤⚔️ #LockwoodandCoSeason2 ...
2        💙🖤⚔️ #LockwoodandCoSeason2 #renewlockwoodandco...
3        RT @vansunshin3: has this been done yet? \n#Sa...
4        @evmcconnell I had that yesterday!!!😋 (bought ...
                               ...                        
86737    This article sums up the whole mess quite nice...
86738    WE WON'T LET YOU! #SaveLockwoodandCo \n\n#Haun...
86739    @hollowgirlhan Classic Rock\n\n#HauntedbyaType...
86740    Hey @carlyleandskull, today's hashtag is just ...
86741    @immzy_ Ok great, now I'm gonna have to finish...
Name: full_text, Length: 86742, dtype: object

In [26]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

working_df['retweet'].value_counts()

True     47149
False    39593
Name: retweet, dtype: int64

In [27]:
import hashlib
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [28]:
if 'anonymized_user' in working_df.columns:
    working_df.drop('anonymized_user', axis=1, inplace=True)
user_df = pd.DataFrame(working_df['user'].unique(), columns=['user'])
user_df['anonymized_user'] = user_df['user'].apply(anonymize)
working_df = working_df.merge(user_df, on='user', how='left')


In [29]:
working_df.columns

Index(['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count',
       'retweet_count', 'hashtags', 'tweet_df', 'retweet', 'anonymized_user'],
      dtype='object')

In [36]:
working_df.to_csv('tweets2205.csv', index=False)

In [27]:
len(working_df)

56513

In [52]:
def get_week_basedontags(x):
    if x in hashtagsweek1:
        return 'week 1'
    elif x in hashtagsweek2:
        return 'week 2'
    elif x in hashtagsweek3:
        return 'week 3'

In [53]:
working_df['week']=working_df['hashtags'].apply(get_week_basedontags)
working_df['week'].value_counts()

week 1    39843
week 2    12768
Name: week, dtype: int64

In [30]:
working_df.to_csv('tweeterhashtags/tweets.csv', index=False)

In [75]:
working_df.value_counts('hashtags')

hashtags
#TogetherForLockwoodandCo    8396
#HauntedWatchParty           7483
#WatchPartyatPortlandRow     6560
#PrimeForLockwoodandCo       6132
#FridayNightatPortlandRow    5916
#HauntedbyaType3             5355
#JustRecklessEnough          3726
#BBCforLockwoodandCo         3097
#AppleTVforLockwoodandCo     2402
#GhostHuntersWatchParty      2046
#DisneyForLockwoodandCo      1246
#BringBackLockwoodandCo       245
dtype: int64